In [2]:
import sys, os
import pandas as pd
import numpy as np

In [3]:
sys.argv[1] = 'trainingSet.csv'
sys.argv[2] = 'testSet.csv'

trainingDataFilename = sys.argv[1]
testDataFilename = sys.argv[2]
# modelIdx = int(sys.argv[3])
modelIdx = 1

In [7]:
def decisionTree(trainingSet, testSet):
    print 'decisionTree'

In [8]:
def bagging(trainingSet, testSet):
    print 'bagging'

In [9]:
def randomForests(trainingSet, testSet):
    print 'randomForests'

In [10]:
trainingSet = pd.read_csv(trainingDataFilename)
testSet = pd.read_csv(testDataFilename)

if modelIdx == 1:
    decisionTree(trainingSet, testSet)
elif modelIdx == 2:
    bagging(trainingSet, testSet)
elif modelIdx == 3:
    randomForests(trainingSet, testSet)
else:
    print 'modelIdx error'

decisionTree
